<font color=magenta size=6>**Energía (Single Point Energy, SPE)**
    
<font color=magenta size=5>**Enrique Mejía Ospino**
    
<font color=magenta size=5>**Química Cuántica Avanzada**
    
<font color=magenta size=5>**Universidad Industrial de Santander**

**<font color=blue> Vamos  atilizar el modulo *pyscf* para desarrollar algunos ejercicios de cálculo mecánico-cuántico. En este Cuaderno de Jupyter realizaremos SPE sobre algunas moleculas sencillas. En el primer se muestra una forma sencilla de calcular la spe de una molécula de agua usando el método de cálculo RHF y la bas ccpdvz.** 

In [2]:
import pyscf
from pyscf import gto, scf, dft, solvent, lo
from pyscf.tools import molden
import psi4
import numpy as np  
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [3]:
mol = gto.Mole()
mol.verbose = 4 # Esta función permite que en el archivo de salida se almacene con más o menos detalles información
# del cálculo y los resultados, con 4 se almacena suficiente detalle
mol.output = './Data/out_h2o' # Este es el nombre del archivo de salida 
mol.atom = '''
O 0 0      0
H 0 -2.757 2.587
H 0  2.757 2.587''' # La molécula
mol.basis = 'ccpvdz' # La base
mol.symmetry = 1 # Algunas características
mol.build()

mf = scf.RHF(mol) # El cálculo

mf.kernel()

overwrite output file: ./Data/out_h2o


-75.25017799679806

**<font color=blue> En este ejemplo se realiza el cálculos utilizando diferentes métodos HF y post-HF.** 

In [4]:
mol = gto.Mole()
mol.output = './Data/out_h2o_MTS'
mol.build(
    atom = '''
      o     0    0       0
      h     0    -.757   .587
      h     0    .757    .587''',  # in Angstrom
    basis = 'ccpvdz',
    symmetry = True,
)
mol.verbose = 4
#mol.output = 'out_h2o_MTS'

# Después de inicializar la molécula, aplicamos (.apply) el método de cálculo SCF o
# post-SCF

mf = mol.apply('RHF').run()
ccobj = mol.apply('CCSD').run()
tdobj = mol.apply('TDHF').run(nstates=5)
print('E(RHF) = %.12f' % mf.e_tot)
print('E(CCSD) = %.12f' % ccobj.e_tot)
tdobj.e_tot

# O

#mf = mol.RHF().run()
#ccobj = mol.CCSD(frozen=2).run()
#tdobj = mol.TDHF().run(nstates=5)
#print('E(RHF) = %.12f' % mf.e_tot)
#print('E(CCSD) = %.12f' % ccobj.e_tot)
#print('E(TDHF) =')
#tdobj.e_tot

overwrite output file: ./Data/out_h2o_MTS
E(RHF) = -76.026765673118
E(CCSD) = -76.240108919632


array([-75.69026295, -75.62543282, -75.59436658, -75.52958448,
       -75.474847  ])

**<font color=blue> En este ejemplo se realiza el cálculo sobre una molécula que se puede cargar en formato .xyz.** 

In [5]:
mol = gto.Mole()
mol.output = './Data/out_glycine'
mol.verbose = 4
mol.atom = open('/home/emejia/pyscf/examples/scf/glycine.xyz').read()
mol.basis = '6-31g*'
mol.build()

mf = scf.RHF(mol)
mf.kernel()
print('E(RHF/631g) = %.12f' % mf.e_tot)
mf = dft.RKS(mol)
mf.kernel()
print('E(DFT/631g) = %.12f' % mf.e_tot)

overwrite output file: ./Data/out_glycine
E(RHF/631g) = -282.761598059558
E(DFT/631g) = -282.125635849190


**<font color=blue> En este ejemplo se realiza el cálculos incluyendo el efecto del solvente.** 

In [5]:
mol = gto.M(atom='''
C        0.000000    0.000000             -0.542500
O        0.000000    0.000000              0.677500
H        0.000000    0.9353074360871938   -1.082500
H        0.000000   -0.9353074360871938   -1.082500
            ''',
            verbose = 3)
mol.output='./Data/ef_solv_out'
mf = scf.RHF(mol)
solvent.ddCOSMO(mf).run()

mf = dft.UKS(mol)
mf.xc = 'b3lyp'
solvent.ddPCM(mf).run()

# Once solvent module is imported, PySCF-1.6.1 and newer supports the .ddCOSMO
# and .ddPCM methods to create solvent model.

mf = mf.ddCOSMO()
# Adjust solvent model by modifying the attribute .with_solvent
mf.with_solvent.eps = 32.613  # methanol dielectric constant
mf.run()

converged SCF energy = -112.354508567381

WARN: ddPCM is an experimental feature. It is still in testing.
Features and APIs may be changed in the future.

converged SCF energy = -112.893027302141  <S^2> = 0  2S+1 = 1
converged SCF energy = -112.892936171586  <S^2> = 9.2370556e-14  2S+1 = 1


<pyscf.solvent._attach_solvent._for_scf.<locals>.SCFWithSolvent at 0x7f5b30eaf310>

In [6]:
from pyscf import gto
from pyscf import __all__

mol = gto.M(atom='''
C        0.000000    0.000000             -0.542500
O        0.000000    0.000000              0.677500
H        0.000000    0.9353074360871938   -1.082500
H        0.000000   -0.9353074360871938   -1.082500
            ''',
            verbose = 3,
            basis = '631g')
#
# Solvent does not respond to the change of electronic structure in vertical
# excitation. The calculation can be started with an SCF with fully relaxed
# solvent and followed by a regular TDDFT method
#
mf = mol.RHF().ddCOSMO().run()
td = mf.TDA()
td.kernel()


#
# Equilibrium solvation allows the solvent rapidly responds to the electronic
# structure of excited states. The system should converge to equilibrium
# between solvent and the excited state of the solute.
#
mf = mol.RHF().ddCOSMO().run()
td = mf.TDA().ddCOSMO()
td.with_solvent.equilibrium_solvation = True
td.kernel()

#
# Switch off the fast solvent
#
td.with_solvent.equilibrium_solvation = False
#td.kernel()

converged SCF energy = -113.816355914777
Excited State energies (eV)
[ 4.45151654 10.29445097 11.70411689]
converged SCF energy = -113.816355914777
Excited State energies (eV)
[ 4.43194897  9.8880442  11.67397748]


In [6]:
import os
import time
import pyscf
from pyscf.tools.mo_mapping import mo_comps

log = pyscf.lib.logger.Logger(verbose=5)
with open('/proc/cpuinfo') as f:
    for line in f:
        if 'model name' in line:
            log.note(line[:-1])
            break
with open('/proc/meminfo') as f:
    log.note(f.readline()[:-1])
log.note('OMP_NUM_THREADS=%s\n', os.environ.get('OMP_NUM_THREADS', None))


for bas in ('3-21g', '6-31g**', 'cc-pVTZ'):
    mol = pyscf.M(atom = '''
c   1.217739890298750 -0.703062453466927  0.000000000000000
h   2.172991468538160 -1.254577209307266  0.000000000000000
c   1.217739890298750  0.703062453466927  0.000000000000000
h   2.172991468538160  1.254577209307266  0.000000000000000
c   0.000000000000000  1.406124906933854  0.000000000000000
h   0.000000000000000  2.509154418614532  0.000000000000000
c  -1.217739890298750  0.703062453466927  0.000000000000000
h  -2.172991468538160  1.254577209307266  0.000000000000000
c  -1.217739890298750 -0.703062453466927  0.000000000000000
h  -2.172991468538160 -1.254577209307266  0.000000000000000
c   0.000000000000000 -1.406124906933854  0.000000000000000
h   0.000000000000000 -2.509154418614532  0.000000000000000
''',
                  basis = bas)
    cpu0 = time.clock(), time.time()

    mf = mol.RHF().run()
    cpu0 = log.timer('C6H6 %s RHF'%bas, *cpu0)

    mymp2 = mf.MP2().run()
    cpu0 = log.timer('C6H6 %s MP2'%bas, *cpu0)

    mymc = mf.CASSCF(6, 6)
    idx_2pz = mo_comps('2pz', mol, mf.mo_coeff).argsort()[-6:]
    mo = mymc.sort_mo(idx_2pz, base=0)
    mymc.kernel(mo)
    cpu0 = log.timer('C6H6 %s CASSCF'%bas, *cpu0)

    mycc = mf.CCSD().run()
    cpu0 = log.timer('C6H6 %s CCSD'%bas, *cpu0)

    mf = mol.RKS().run(xc='b3lyp')
    cpu0 = log.timer('C6H6 %s B3LYP'%bas, *cpu0)

    mf = mf.density_fit().run()
    cpu0 = log.timer('C6H6 %s density-fit RHF'%bas, *cpu0)

model name	: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz
MemTotal:        8277196 kB
OMP_NUM_THREADS=None

converged SCF energy = -229.412681529859
    CPU time for C6H6 3-21g RHF      1.33 sec, wall time      0.19 sec
E(MP2) = -229.946761429545  E_corr = -0.534079899685633
    CPU time for C6H6 3-21g MP2      0.34 sec, wall time      0.05 sec
CASSCF energy = -229.489635419041
CASCI E = -229.489635419041  E(CI) = -6.46551128162392  S^2 = 0.0000000
    CPU time for C6H6 3-21g CASSCF      6.50 sec, wall time      0.85 sec
E(CCSD) = -229.9883806714248  E_corr = -0.5756991415653785
    CPU time for C6H6 3-21g CCSD     51.61 sec, wall time      6.95 sec
converged SCF energy = -230.81975465152
    CPU time for C6H6 3-21g B3LYP     22.77 sec, wall time      4.60 sec
converged SCF energy = -230.820000977398
    CPU time for C6H6 3-21g density-fit RHF     22.27 sec, wall time      4.28 sec
converged SCF energy = -230.708074851859
    CPU time for C6H6 6-31g** RHF      6.61 sec, wall time      0.85

**<font color=blue> En este ejemplo se realiza el cálculos PSE utlizando el modulo *PSI4*.** 

In [17]:
# Inicialización Psi4 =================================================================
import psi4
psi4.core.clean()
psi4.core.clean_options()
psi4.set_memory('2000 MB')  # Memoria que sera utilizada en el cálculo
psi4.set_num_threads(4)    # Cantidad de hilos a utilzar, tener en cuenta que un núcleo soporta 24 hilos. 
                          
psi4.core.set_output_file('./Data/pse1.dat', False) 
# Este comando nos permite guardar el archivo de salida en alguna parte del computador.

**<font color=cian> Se carga la molécula, en este ejemplo se hará desde un archivo en formato *xyz*:** 

In [18]:
# Inicialización de los cálculos ================================================

# En este ejemplo se carga la molécula de un archivo en formato .xyz
f = open('./molecules/benzene.xyz')
molecula = psi4.geometry(f.read())
molecula.reset_point_group('c1') # Simetría!

**<font color=red> Si lo desea, puede dibujar la molécula, existen algunos modulos de visualización, aquí empleamos el modulo *fortecubeview*.** 

In [19]:
import fortecubeview as fcv # Visualizar moléculas
fcv.geom(molecule = molecula)

Renderer(camera=OrthographicCamera(bottom=-5.0, children=(DirectionalLight(color='white', intensity=0.5, posit…

**<font color=blue> Realizamos el cálculo, seleccionando el métod y las bases inicialmente, en este caso vamos a emplear dos *bases* diferentes.** 

In [20]:
# Métodos de cálculos.
metodo1 = 'scf/3-21G'  # Métodos de cálculos.
metodo2 = 'scf/6-311G*'

E0_1, wfn1 = psi4.energy(metodo1, molecule = molecula, return_wfn = True)  # cálculo de energía usando metodo1
E0_2, wfn2 = psi4.energy(metodo2, molecule = molecula, return_wfn = True)  # cálculo de energía usando metodo2

Eopt1, wfn_opt1 = psi4.optimize(metodo1, molecule = molecula, return_wfn = True)
psi4.driver.molden(wfn_opt1, './Data/molecula1.molden') # archivo de salida en formato .molden.
psi4.driver.molden(wfn2, './Data/molecula2.molden') # archivo de salida en formato .molden.

# Impresión de Resultados =================================================================
print('Energías en las posiciones iniciales con los dos métodos:')
print()
print('Energía con el ' + metodo1 + ' es ' + str(E0_1) + ' Hatrees')
print('Energía con el ' + metodo2 + ' es ' + str(E0_2) + ' Hatrees')
print('Energía de optimización con el ' + metodo1 + ' es ' + str(Eopt1) + ' Hatrees')
print()

Optimizer: Optimization complete!
Energías en las posiciones iniciales con los dos métodos:

Energía con el scf/3-21G es -229.4176723388107 Hatrees
Energía con el scf/6-311G* es -230.7417261352083 Hatrees
Energía de optimización con el scf/3-21G es -229.41937144999395 Hatrees



In [25]:
import psi4
hartree2ev = psi4.constants.hartree2ev

psi4.set_output_file("benzeno_out.dat", False)

benz = psi4.geometry("""
                        pubchem:80-69-3""")

psi4.set_options({"REFERENCE"                : "RHF",
                  "MAX_ENERGY_G_CONVERGENCE" : 8,
                  "BASIS"                    : "STO-3G",
                  "DF_BASIS_SCF"             : "CC-PVDZ-RI"})

psi4.optimize('scf')

psi4.set_options({"REFERENCE"    : "RHF",
                  "BASIS"        : "CC-PVDZ",
                  "DF_BASIS_SCF" : "CC-PVDZ-JKFIT"})

e_sing_rhf = psi4.energy('scf')

benz.set_multiplicity(3)

psi4.set_options({"REFERENCE" : "ROHF"})
e_trip_rohf = psi4.energy('scf')
psi4.set_options({"REFERENCE" : "UHF"})
e_trip_uhf  = psi4.energy('scf')

vertical_uhf  = hartree2ev * (e_trip_uhf  - e_sing_rhf)
vertical_rohf = hartree2ev * (e_trip_rohf - e_sing_rhf)
psi4.core.print_out("\nSinglet-Triplet gap (vertical, UHF)   = %8.2f eV\n" % vertical_uhf)
psi4.core.print_out("\nSinglet-Triplet gap (vertical, ROHF)  = %8.2f eV\n" % vertical_rohf)
print("\nSinglet-Triplet gap (vertical, UHF)   = %8.2f eV\n" % vertical_uhf)
print("\nSinglet-Triplet gap (vertical, ROHF)  = %8.2f eV\n" % vertical_rohf)
#e_sing_rhf

	Searching PubChem database for 80-69-3 (single best match returned)
	Found 1 result(s)
Optimizer: Optimization complete!

Singlet-Triplet gap (vertical, UHF)   =     4.14 eV


Singlet-Triplet gap (vertical, ROHF)  =     4.39 eV



In [ ]:
e_sing_rhf, e_trip_rohf, e_trip_uhf, vertical_uhf, vertical_rohf

In [31]:
import numpy as np

import psi4

from psi4.driver.procrouting.response.scf_response import tdscf_excitations
from psi4 import frequencies

psi4.core.set_output_file("moxy.out")

moxy = psi4.geometry("""0 1
C  0.152133 -0.035800  0.485797
C -1.039475  0.615938 -0.061249
C  1.507144  0.097806 -0.148460
O -0.828215 -0.788248 -0.239431
H  0.153725 -0.249258  1.552136
H -1.863178  0.881921  0.593333
H -0.949807  1.214210 -0.962771
H  2.076806 -0.826189 -0.036671
H  2.074465  0.901788  0.325106
H  1.414895  0.315852 -1.212218
""", name="(S)-methyloxirane")

psi4.set_options({
    'save_jk': True,
})

e, wfn = psi4.energy("HF/cc-pvdz", return_wfn=True, molecule=moxy)
res = tdscf_excitations(wfn, states=8, triplets="also")

# get poles and residues to plot OPA and ECD spectra
poles = [r["EXCITATION ENERGY"] for r in res]
opa_residues = [np.linalg.norm(r["LENGTH-GAUGE ELECTRIC DIPOLE TRANSITION MOMENT"])**2 for r in res]
ecd_residues = [r["LENGTH-GAUGE ROTATORY STRENGTH"] for r in res]

#opa_spectrum = spectrum(poles=poles, residues=opa_residues, gamma=0.01, out_units="nm")
#ecd_spectrum = spectrum(poles=poles, residues=ecd_residues, kind="ECD", gamma=0.01, out_units="nm")
poles
opa_residues
res
e
wfn
psi4.molden(wfn,"./Data/wfn_angular.molden")

In [1]:
from pyscf import gto, scf
from pyscf.geomopt.berny_solver import optimize
mol = gto.M(atom='N 0 0 0; N 0 0 1.2', basis='ccpvdz')
mf = scf.RHF(mol)
mol_eq = optimize(mf)
print(mol_eq.atom_coords())

/home/emejia/psi4conda/envs/psi4/lib/python3.7/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'



Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N   0.000000   0.000000   0.000000    0.000000  0.000000  0.000000
   N   0.000000   0.000000   1.200000    0.000000  0.000000  0.000000
converged SCF energy = -108.914051975052
--------------- SCF_Scanner gradients ---------------
         x                y                z
0 N    -0.0000000000    -0.0000000000    -0.3121321959
1 N     0.0000000000     0.0000000000     0.3121321959
----------------------------------------------
cycle 1: E = -108.914051975  dE = -108.914  norm(grad) = 0.441422

Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   N   0.000000   0.000000   0.079377    0.000000  0.000000  0.079377
   N   0.000000   0.000000   1.120623    0.000000  0.000000 -0.079377

WARN: Large deviations found between the input molecule and the molecule from chkfile
Initial guess 

In [2]:
pip install -U pyberny

Note: you may need to restart the kernel to use updated packages.
